In [108]:
import random
from src.bandit.algorithms.standard import EpsilonGreedy
from src.bandit.arms.bernoulli import BernoulliArm
from src.bandit.simulation.monte_carlo import test_algorithm

In [110]:
EpsilonGreedy(0.2)

TypeError: EpsilonGreedy.__init__() missing 2 required positional arguments: 'counts' and 'values'

In [53]:
discounts = [0.4, 0.3, 0.2, 0.1, 0.0] # discount offered in each arm
means = [0.015, 0.01, 0.008, 0.005, 0.005] # probability of conversion for each corresponding discount
n_arms = len(means)
random.shuffle(means)
arms = [BernoulliArm(i) for i in means]

In [ ]:
for epsilon in [0.1, 0.2, 0.3, 0.4]:
    algo = EpsilonGreedy(epsilon,[],[])
    algo.initialize(n_arms)
    results = test_algorithm(algo, arms, 5000, 250)